# Running SoS workflows on cluster systems

* **Difficulty level**: intermediate
* **Time need to lean**: 30 minutes or less
* **Key points**:
  * SoS workflows recognizes cluster systems (PBS, Slurm, LSF etc) and make use of allocated nodes as workers
  * Option `-j` will be ignored, but option `-q none` is recommended to suppress external tasks.  

## Use of multiple workers to execute SoS workflows

SoS uses multiple worker processes to execute steps, substeps, and subworkflows. By default, SoS creates `n/2` workers on a local computer where `n` is the number of CPU cores, although it limits the default number of workers to 8 on computers with more than 16 cores because those computers are most likely shared by multiple users.

The number of workers used by SoS can be controlled by option `-j`, where `-j 4` creates 4 workers so that you will see 5 sos processes (1 master and 4 worker) when you execute a workflow with command

```
sos run script -j 4
```

If you have multiple machines with a shared file system, it is possible to start workers on remote machines by specifying the name of the machine and number of processes with an extended version of option `-j`. For example

```
sos run script -j 4 pc1:4 pc2:4
```

will create 12 workers, 4 on localhost, 4 on `pc1` and 4 on `pc2`, where `pc1` and `pc2` can be name or IP address of machines, or an aliaes defined in SoS configuration files. This usage of option `-j` is, however, rarely used because personal computers rarely share file systems with remote computers.

## Running SoS workflows on clusters

A more likely scenario for multiple computers with a shared file system is computer clusters. In this case, multiple nodes can be allocated to execute a single SoS workflow, effectively creating a "mini-cluster" inside a cluster system. The master SoS process will be started on one of the computing nodes and start multiple workers on the head and the rest of the allocated computing nodes.

<p align="center">
  <img src="https://vatlab.github.io/sos-docs/doc/media/cluster_execution.jpg">
</p>

Because the `sos` command does not know the exact computing node allocated, SoS automaticaly detects the scheduler of a PBS-like cluster system and set `-j` accordingly. So a typical way to execute a large workflow on multiple nodes would be the creation of a wrapper script as follows:

```sh
#!/bin/bash
#PBS -N workflow
#PBS -l nodes=4:ppn=8
#PBS -l walltime=10:0:0
#PBS -l mem=10GB
#PBS -m ae

cd /home/jovyan/projects
sos run workflow.sos -q none
```

and submit to the cluster with appropriate command (e.g. `qsub workflow.sh`). Note that this script is used for PBS cluster and the syntax for other cluster systems such as LSF and SLURM will vary.

Note that

1. Option `-j` is not needed because it would be automatically created by SoS according to environment-specific variables such as `PBS_NODEFILE`.
2. Option `-q none` is recommended to execute external tasks as regular substeps.


## Further reading

* [Running external tasks](tasks.html) for how to define and execute tasks externally, and the use of option `-q none`.